# lifespan

> Shared objects for FastAPI app

In [ ]:
# | default_exp api/lifespan

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import os
from contextlib import asynccontextmanager

from fastapi import FastAPI
from openai import AsyncOpenAI

In [ ]:
def get_openai_compat_clients(store):
    new_store = store.copy()
    env_vars = [k for k, v in os.environ.items()]
    new_store["openai"]["openai"] = AsyncOpenAI()

    for k in env_vars:
        if k.startswith("OPENAI_API_KEY_"):
            key = k.split("_")[-1]
            new_store["openai"][key] = AsyncOpenAI(api_key=os.environ[k])

    return new_store

In [ ]:
# | export


@asynccontextmanager
async def lifespan(app: FastAPI):
    app.state.clients = {}
    app.state.clients = get_openai_compat_clients(app.state.clients)
    yield
    app.state.clients.clear()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()